## 폐암 수술 환자의 생존율 예측

In [1]:
import os

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# 필요한 라이브러리 불러옴
import numpy as np

print(torch.__version__)

import pandas as pd

# 실행할 때마다 같은 결과를 출력하기 위해 설정하는 부분
np.random.seed(3)
torch.manual_seed(3)

'''
# 준비된 수술 환자 데이터를 불러옴
Data_set = np.loadtxt("../dataset/ThoraricSurgery.csv", delimiter=',')

# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장
X = Data_set[:, 0:17]
Y = Data_set[:, 17]
'''

# 준비된 수술 환자 데이터를 불러옴 by using pandas
#df = pd.read_csv("../dataset/ThoraricSurgery.csv",
#                      names=["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "class"])
df = pd.read_csv("../dataset/ThoraricSurgery.csv", header=None)
print(df.head())
df.rename(columns={17:"class"}, inplace=True)
print(df.head())

# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장
X = df.drop(['class'], axis=1, inplace=False).values
Y = df['class'].values

1.6.0
    0   1     2     3   4   5   6   7   8   9   10  11  12  13  14  15  16  17
0  293   1  3.80  2.80   0   0   0   0   0   0  12   0   0   0   1   0  62   0
1    1   2  2.88  2.16   1   0   0   0   1   1  14   0   0   0   1   0  60   0
2    8   2  3.19  2.50   1   0   0   0   1   0  11   0   0   1   1   0  66   1
3   14   2  3.98  3.06   2   0   0   0   1   1  14   0   0   0   1   0  80   1
4   17   2  2.21  1.88   0   0   1   0   0   0  12   0   0   0   1   0  56   0
     0  1     2     3  4  5  6  7  8  9  10  11  12  13  14  15  16  class
0  293  1  3.80  2.80  0  0  0  0  0  0  12   0   0   0   1   0  62      0
1    1  2  2.88  2.16  1  0  0  0  1  1  14   0   0   0   1   0  60      0
2    8  2  3.19  2.50  1  0  0  0  1  0  11   0   0   1   1   0  66      1
3   14  2  3.98  3.06  2  0  0  0  1  1  14   0   0   0   1   0  80      1
4   17  2  2.21  1.88  0  0  1  0  0  0  12   0   0   0   1   0  56      0


In [50]:
class Model(nn.Module):
    """
    """
    def __init__(self):
        super(Model, self).__init__()
        
        self.input_size = 17
        
        self.layers = nn.Sequential(
            nn.Linear(in_features=self.input_size, out_features=30, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=30, out_features=1, bias=True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        out = self.layers(x)
        return out
        

In [51]:
model = Model()
model

Model(
  (layers): Sequential(
    (0): Linear(in_features=17, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

In [52]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print("device :", device)
epochs = 500
batch_size = 64

x = torch.from_numpy(X).type(torch.FloatTensor)
y = torch.from_numpy(Y).type(torch.LongTensor)

dataset = TensorDataset(x, y)       
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)  
    
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
#criterion = nn.BCEWithLogitsLoss() # Sigmoid + BCELoss

#    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
#                                 lr=config.learning_rate, betas=(0.9, 0.98), eps=1e-09)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.5, 0.999))
        
for epoch in range(epochs):
    phase = 'train'
    for k, [inputs, targets] in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(phase=='train'):
            output = model(inputs)
            
            targets = targets.type_as(output)
            loss = criterion(output, targets)
            #print(loss)            
            
            loss.backward(retain_graph=True)
            optimizer.step()

device : cpu


In [53]:
prediction = model(x)
#prediction = torch.sigmoid(prediction)
prediction.detach().numpy()

array([[0.52393085],
       [0.56837535],
       [0.723369  ],
       [0.6149242 ],
       [0.51904935],
       [0.5067892 ],
       [0.5065459 ],
       [0.54370856],
       [0.5427494 ],
       [0.5210081 ],
       [0.5227097 ],
       [0.5390689 ],
       [0.52811724],
       [0.52230406],
       [0.6194684 ],
       [0.53620976],
       [0.5454474 ],
       [0.5316139 ],
       [0.5243564 ],
       [0.54765254],
       [0.5465744 ],
       [0.56838787],
       [0.6049236 ],
       [0.5520054 ],
       [0.5714037 ],
       [0.53625304],
       [0.6059196 ],
       [0.53918016],
       [0.5531917 ],
       [0.5314384 ],
       [0.5423662 ],
       [0.5608119 ],
       [0.59901   ],
       [0.54690605],
       [0.5802605 ],
       [0.5064137 ],
       [0.5294952 ],
       [0.5615675 ],
       [0.5422143 ],
       [0.5455523 ],
       [0.5443995 ],
       [0.5609928 ],
       [0.52973384],
       [0.5408163 ],
       [0.5268213 ],
       [0.54343194],
       [0.5338084 ],
       [0.537